In [10]:
# read df from csv file
import pandas as pd
import os

# get current working directory
cwd = os.getcwd()

df = pd.read_csv(cwd + '/output_data/polish_budget.csv', sep=";")

# create treemap using plotly
import plotly.graph_objects as go


# color palette
# change colors to palettable
import palettable as pltt

# set color palette
color_scale = pltt.tableau.Tableau_20.hex_colors


fig = go.Figure()

# add trace
fig.add_trace(go.Treemap(
    ids= df['id'],
    labels = df['super_label'],
    parents = df['parent'],
    values = df['Plan'],
    # add 3 columns to customdata
    textinfo = "label+value+percent parent+percent entry",
    customdata= df[['Treść_część', 'Treść_dział', 'Treść', 'label']],
    pathbar=dict(
        visible=True
    ), 
    texttemplate = '%{customdata[2]}<br>' + '%{customdata[3]}',
    textposition = 'middle center',
))

# add title
fig.update_layout(
    title_text = 'Planowane wydatki budżetowe 2023',
    # add padding
    margin = dict(t=50, l=25, r=25, b=25),
)

fig.update_traces(branchvalues = 'total')

# create dynamic hovertemplate
# show hovertemplate = '<b>%{customdata[2]}<br></b>' + 'Kwota: %{value}<br>' + '%{percentParent: .0%} of %{parent}<br>'
                #   + '%{percentRoot: .0%} of total<br>'
                #   + 'Treść: %{label}<br>' + 'Część: %{customdata[0]}<br>' + 'Dział: %{customdata[1]}<br>' + '<extra></extra>
                # if parent is not total, else show only % of root

template = [
    '<b>%{customdata[2]}<br></b>' + 'Kwota: %{customdata[3]}<br>' + '%{percentParent: .2%} of %{parent}<br>'
    + '%{percentRoot: .0%} of total<br>'
    + '<extra></extra>' if parent != 'total' else
    '<b>%{customdata[2]}<br></b>' + 'Kwota: %{customdata[3]}<br>' + '%{percentRoot: .2%} of total<br>'
    + '<extra></extra>'
    for parent in df['parent']
]

# dynamic template
def dynamic_text_template(depth):
    # if depth = 3 -> show only label
    if depth == 3:
        # create template
        template = '%{customdata[3]}'
        return template
    
    # if depth = 2 -> show label + % of parent
    elif depth == 2:
        # create template
        template = '%{customdata[3]}<br>' + '%{percentParent: .2%} of %{parent}<br>'
        return template
    
    # if depth = 1 -> show label + % of parent + % of root
    elif depth == 1:
        # create template
        template = '%{customdata[3]}<br>' + '%{percentParent: .2%} of %{parent}<br>' + '%{percentRoot: .2%} of total<br>'
        return template
        


# use template to create hovertemplate
fig.update_traces(
    hovertemplate = template
)

# size 1200x1200
fig.update_layout(
    width = 1200,
    height = 1200,
    margin = dict(t=50, l=25, r=25, b=25),
    # paper_bgcolor = 'rgba(0,0,0,0)',
    plot_bgcolor = 'rgba(0,0,0,0)'
)


# set default text template
fig.update_traces(
    texttemplate = '%{customdata[3]}',
    textposition = 'middle center'
)

# detailed texttemplate on tile -> show label, value, % of parent, % of root, create as dict

fig.layout.update(
    # add text above buttons
    annotations=[
        
        go.layout.Annotation(
            text="<b>Depth</b>",
            showarrow=False,
            x=0.9,
            y=1.07,
            yref="paper",
            align="left"
        ),

        # add clickable link to github repo
        go.layout.Annotation(
            text="<b>Author</b>: <a href='https://github.com/seszele64/'>seszele64</a>",
            showarrow=False,
            x=0.95,
            y=1.04,
            yref="paper",
            align="left"
        )

    ],

    # insert another text with author name and link to github and github icon
    images=[
        go.layout.Image(
            source="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png",
            yref="paper",
            x=0.96,
            y=1.05,
            sizex=0.03,
            sizey=0.03,
        )
    ],


    updatemenus=[
        go.layout.Updatemenu(
            active=0,
            type="buttons",
            direction="left",
            buttons=list(
                [
                    dict(args=[{"maxdepth": 4, "texttemplate": "%{customdata[3]}"}, None], label="4", method="restyle"),
                    dict(args=[{"maxdepth": 3, "texttemplate": "%{customdata[2]}<br> %{customdata[3]}"}, None], label="3", method="restyle"),
                    dict(args=[{
                        "maxdepth": 2,
                        "texttemplate": "%{customdata[2]}<br>" + "%{customdata[3]}<br>" + "%{percentParent: .2%} of %{parent}<br>"
                    }, None], label="2", method="restyle"),
                ]
            ),
            pad={"t": 20},
            showactive=True,
            x=0.9,
            xanchor="left",
            y=1.1,
            yanchor="top", 
            # background of buttons
            bgcolor = 'rgba(255, 255, 255, 0.521)',


        ),
    ]
)


# change left top text of every tile to label + value
fig.update_layout(
    template = 'plotly_white',
    treemapcolorway = color_scale,
    # uniformtext = dict(minsize = 12, mode = 'hide')
)


# change tiling to binary
fig.update_traces(tiling=dict(packing='binary', pad = 5))


# ! todo: - change total total to "Budżet państwa"


fig.show()